In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
PATH = 'google_books_1299.csv'

def get_data(path_to_data):
#read the csv
    data = pd.read_csv(f'{path_to_data}',index_col=0)
#make the title to lower characters
    data["title"] = data["title"].str.lower()
#drop the missing value
    data = data.dropna()
#create index from "0"
    data.index = [i for i in range(0,len(data))]
    return data

In [6]:
#get data
data = get_data(PATH)

In [7]:
#transfer the object in the generes columns to string
data['generes'] = data['generes'].astype('string')

In [8]:
#import
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

In [1]:
#This framework provides an easy method to compute dense vector representations for sentences.The models are based on transformer networks like BERT.Text is embedding in vector space such that similar text is close and can efficiently be found using cosine similarity.
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/Users/su/opt/anaconda3/envs/project/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def recommender(genres, data):
    book_dic = {} #create a dictionary
    
    for index, value in data.iterrows(): #.iterrows: Iterate over DataFrame rows as (index, Series) pairs. -read the DataFrame row by row
        genres_embeddings = model.encode([genres]) #encode genres
        value_embeddings = model.encode([value[9]])#encode the values in genres
        cosine = 1 - spatial.distance.cosine(genres_embeddings, value_embeddings) #1- cosine distance between the two =similarity
        if cosine > 0.7:
            book_dic[index] = cosine
        else:
            continue
    if book_dic and len(list(book_dic.keys()))>=5: #condition when there are more than 5 qualified books 
        book_dic = dict(sorted(book_dic.items(), key=lambda x: x[1], reverse=True)) #list the books from high to low cosine value
        book_indexs = list(book_dic.keys())[:5] #show the top 5 books
        recommendation = pd.DataFrame(columns=['ISBN','title']) #create colunms and the format shown is ISBN,title
        for book_index in book_indexs:
            book_ISBN = data['ISBN'].iloc[book_index] #map the 'ISBN' according to the book index
            book_title = data['title'].iloc[book_index] #map the 'title' according to the book index
            recommendation=recommendation.append({'ISBN' : book_ISBN , 'title' : book_title} , ignore_index=True) #append the list     
        return recommendation
    elif book_dic:
        print("We can only give you less than 5 recommendations") #condition when there are less than 5 qualified books 
        book_dic = dict(sorted(book_dic.items(), key=lambda x: x[1], reverse=True))
        book_indexs = list(book_dic.keys())
        recommendation = pd.DataFrame(columns=['ISBN','title'])
        for book_index in book_indexs:
            book_ISBN = data['ISBN'].iloc[book_index]
            book_title = data['title'].iloc[book_index]
            recommendation=recommendation.append({'ISBN' : book_ISBN , 'title' : book_title} , ignore_index=True)      
        return recommendation
        
    else:
        recommendation = "Sorry, we do not have any recommentation for you."
        return recommendation
        
def Content_based_recommender(data):

    data = data[data['generes'].notna()] #drop the missing value
    data['generes'] = data['generes'].str.lower()

    print(f"What type of genre do you like? \n\nYou can choose from the following:\n\n{set(data['generes'])}")
    genres = input().lower()
    
    recommendations = recommender(genres, data)

    return recommendations

In [ ]:
Content_based_recommender(data)